<a href="https://colab.research.google.com/github/Jideco/pyTorch_deep_learning/blob/main/PyTorch_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=8284c13b5c70bfb49310c9c30a55a5500218413fa91b5ab1f6c878b71cb6c544
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
from torchvision import transforms

In [29]:
from PIL import Image
import numpy as np

In [ ]:
class SimpleClothingClassifier(nn.Module):
    def __init__(self):
        super(SimpleClothingClassifier, self).__init__()

        # 1. Convolutional Layer: 3 input channels (RGB), 32 filters, 3x3 kernel
        # Input: (3, 200, 200) -> Output: (32, 198, 198)
        self.conv2d = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=1, padding=0)

        # ReLU Activation
        self.relu = nn.ReLU()

        # 2. Max Pooling Layer: 2x2 pooling size
        # Input: (32, 198, 198) -> Output: (32, 99, 99)
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2))

        # 3. Flatten the multi-dimensional result into a vector
        # Calculation: 32 channels * 99 width * 99 height = 313,632 elements
        self.flatten = nn.Flatten()

        # 4. Dense (Linear) Layer with 64 neurons
        self.fc1 = nn.Linear(32 * 99 * 99, 64)

        # 5. Output Layer: 1 neuron for binary classification
        self.output_layer = nn.Linear(64, 1)

        # Sigmoid activation for binary classification probabilities
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply Convolution -> ReLU -> MaxPool
        x = self.conv2d(x)
        x = self.relu(x)
        x = self.max_pool(x)

        # Flatten for the linear layers
        x = self.flatten(x)

        # First linear layer with ReLU
        x = self.fc1(x)
        x = self.relu(x)

        # Output layer with Sigmoid
        x = self.output_layer(x)
        x = self.sigmoid(x)
        return x

def make_model(learning_rate=0.01):
    model = SimpleClothingClassifier()
    # Move model to available device (CPU/GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Use Adam optimizer for the new model parameters
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    return model, optimizer

In [14]:
class SimpleClothingClassifier(nn.Module):
    def __init__(self):
        super(SimpleClothingClassifier, self).__init__()

        # 1. Convolutional Layer: 3 input channels, 32 filters, 3x3 kernel
        # Output spatial size calculation: (200 - 3)/1 + 1 = 198
        self.conv2d = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=1, padding=0)
        self.relu = nn.ReLU()

        # 2. Max Pooling Layer: 2x2 size reduces 198x198 to 99x99
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2))

        # 3. Flattening for the linear layer
        # Size: 32 channels * 99 width * 99 height = 313,632
        self.flatten = nn.Flatten()

        # 4. Dense Layer: 64 neurons
        self.fc1 = nn.Linear(32 * 99 * 99, 64)

        # 5. Output Layer: 1 neuron (Binary Classification)
        # Note: Sigmoid is REMOVED here because we use BCEWithLogitsLoss
        self.output_layer = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv2d(x)
        x = self.relu(x)
        x = self.max_pool(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)

        # Return raw logits (the raw numbers before sigmoid)
        x = self.output_layer(x)
        return x

def make_model():
    model = SimpleClothingClassifier()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # LOSS FUNCTION: Numerically stable for binary tasks
    criterion = nn.BCEWithLogitsLoss()

    # OPTIMIZER: SGD with lr=0.002 and momentum=0.8
    optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

    return model, optimizer, criterion


In [17]:
model, optimizer, criterion = make_model()

In [18]:
!pip install torchsummary
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
           Flatten-4               [-1, 313632]               0
            Linear-5                   [-1, 64]      20,072,512
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 23.93
Params size (MB): 76.57
Estimated Total Size (MB): 100.96
----------------------------------------------------------------


In [22]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [38]:
import os
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [39]:
from torch.utils.data import DataLoader

train_dataset = Dataset(
    data_dir='/content/drive/MyDrive/Hair_dataset/data/train',
    transform=train_transforms
)

test_dataset = Dataset(
    data_dir='/content/drive/MyDrive/Hair_dataset/data/test',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.1248, Acc: 0.9613, Val Loss: 0.7535, Val Acc: 0.7015
Epoch 2/10, Loss: 0.0896, Acc: 0.9763, Val Loss: 1.0292, Val Acc: 0.6965
Epoch 3/10, Loss: 0.0612, Acc: 0.9850, Val Loss: 1.0388, Val Acc: 0.7264
Epoch 4/10, Loss: 0.0275, Acc: 0.9988, Val Loss: 1.3841, Val Acc: 0.6667
Epoch 5/10, Loss: 0.0565, Acc: 0.9800, Val Loss: 0.9849, Val Acc: 0.7214
Epoch 6/10, Loss: 0.0249, Acc: 0.9950, Val Loss: 0.9485, Val Acc: 0.7562
Epoch 7/10, Loss: 0.0145, Acc: 1.0000, Val Loss: 1.0008, Val Acc: 0.7512
Epoch 8/10, Loss: 0.0073, Acc: 1.0000, Val Loss: 1.0785, Val Acc: 0.7562
Epoch 9/10, Loss: 0.0059, Acc: 1.0000, Val Loss: 1.1537, Val Acc: 0.7313
Epoch 10/10, Loss: 0.0046, Acc: 1.0000, Val Loss: 1.1084, Val Acc: 0.7463


In [35]:
import statistics

data = [0.1248, 0.0896,0.0612, 0.0275, 0.0565, 0.0249, 0.0145, 0.0073, 0.0059, 0.0046]

# Sample standard deviation (uses N-1 in denominator)
sample_std_dev = statistics.stdev(data)
print(f"Sample std dev: {sample_std_dev}")

# Population standard deviation (uses N in denominator)
population_std_dev = statistics.pstdev(data)
print(f"Population std dev: {population_std_dev}")


Sample std dev: 0.0406399311023038
Population std dev: 0.03855442387067922


In [37]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [40]:
num_epochs = 20
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/20, Loss: 0.7011, Acc: 0.6292, Val Loss: 0.6811, Val Acc: 0.6418
Epoch 2/20, Loss: 0.6389, Acc: 0.5868, Val Loss: 0.6717, Val Acc: 0.5970
Epoch 3/20, Loss: 0.6450, Acc: 0.6117, Val Loss: 0.6227, Val Acc: 0.6368
Epoch 4/20, Loss: 0.5882, Acc: 0.6442, Val Loss: 0.6838, Val Acc: 0.5871
Epoch 5/20, Loss: 0.6101, Acc: 0.6529, Val Loss: 0.6999, Val Acc: 0.6368
Epoch 6/20, Loss: 0.5628, Acc: 0.7041, Val Loss: 0.6972, Val Acc: 0.5473
Epoch 7/20, Loss: 0.6371, Acc: 0.5805, Val Loss: 0.6963, Val Acc: 0.6468
Epoch 8/20, Loss: 0.5791, Acc: 0.6816, Val Loss: 0.6915, Val Acc: 0.6816
Epoch 9/20, Loss: 0.5686, Acc: 0.6954, Val Loss: 0.6291, Val Acc: 0.6468
Epoch 10/20, Loss: 0.5943, Acc: 0.6742, Val Loss: 0.5888, Val Acc: 0.6915
Epoch 11/20, Loss: 0.5338, Acc: 0.7366, Val Loss: 0.7144, Val Acc: 0.6567
Epoch 12/20, Loss: 0.5468, Acc: 0.7416, Val Loss: 0.5347, Val Acc: 0.7463
Epoch 13/20, Loss: 0.5351, Acc: 0.7478, Val Loss: 0.9164, Val Acc: 0.5323
Epoch 14/20, Loss: 0.6631, Acc: 0.6242, Val Los